# Inspect and Verify that the Source Table is reasonable
Michael Wood-Vasey

In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

In [ ]:
object_reader = 'dc2_object_run1.2i'
source_reader = 'dc2_source_run1.2i'

In [ ]:
dpdd_parquet_file = 'dpdd_dc2_object_run1.2i.parquet'
dpdd_parquet_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.2i/object_catalog_new'
object_table_parquet_file = os.path.join(dpdd_parquet_dir, dpdd_parquet_file)

source_table_parquet_file = '../test_src_visit_241985.parquet'

In [ ]:
object_table = pd.read_parquet(object_table_parquet_file)
source_table = pd.read_parquet(source_table_parquet_file)

In [ ]:
print(len(object_table), len(source_table))

In [ ]:
source_table.columns

In [ ]:
source_table.dtypes

In [ ]:
plt.scatter(source_table['mag'], source_table['mag_err'])
plt.ylim(0, 0.5)

In [ ]:
# Match the object ID and check that we really got the right thing in the source table ID

In [ ]:
print(source_table[['id', 'objectId']])

In [ ]:
source_table.dtypes

In [ ]:
source_table[['visit', 'sourceId', 'objectId', 'fluxmag0']].dtypes

In [ ]:
resid = source_table['mag'] - (-2.5*np.log10(source_table['slot_PsfFlux_instFlux']/source_table['fluxmag0']))

In [ ]:
print(resid)

In [ ]:
for s in source_table:
    print(s)

In [ ]:
source_table['slot_PsfFlux_instFlux']

In [ ]:
source_table['slot_ApFlux_instFlux']